## Setup and Installation

[Full instructions of Jupyter lab setup](https://github.com/TrelisResearch/install-guides/blob/main/jupyter-lab-setup.md)

In [ ]:
# !pip install requests

Note: If you are using Apple Silicon (M1) Mac, make sure you have installed a version of Python that supports arm64 architecture; Otherwise, while installing it will build the llama.ccp x86 version which will be 10x slower on Apple Silicon (M1) Mac. For example:

In [ ]:
arm64path = "Miniforge3-MacOSX-arm64.sh"
if os.path.exists(arm64path):
    print("Version of Python that supports arm64 architecture already exists!")
else:
    print("Uncomment the next block of code and install python.")

## Install Llama.cpp (only need to do this once)
The instructions below are for Macs with an M1 chip.
For other operating systems, comment out those cells and get instructions [here](https://github.com/TrelisResearch/llamacpp-install-basics/blob/main/instructions.md).

In [ ]:
# Download the model file
model_name = 'TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'
pure_name = model_name.split('/')[-1]
print("Pure name of model is: ", pure_name)

parts = model_name.split('/')
model_path = f"{parts[0]}/{parts[1]}"

print("Model path is: ", model_path)

In [ ]:
if not os.path.exists('llama.cpp'):
    print("Cloning llama.cpp...")
    !git clone https://github.com/ggerganov/llama.cpp
    %cd llama.cpp

    print("Compiling for Mac with M1 chip...")
    !LLAMA_METAL=1 make
    print("Compilation completed!")
            
    %cd ../
else:
    print("llama.cpp has already been cloned into this directory!")

### set directory to llama.cpp

In [ ]:
%cd llama.cpp

if not os.path.exists(pure_name):
    !wget https://huggingface.co/{model_name}
else:
    print(f"{pure_name} already exists!")

%cd ../

Set the default value for context_length to High Speed (4096)

In [ ]:
context_length = 2048
max_doc_length = int(0.75 * context_length)
max_doc_tokens = max_doc_length
n_predict = int(0.2 * context_length)

In [ ]:
import sys
import subprocess
import threading
import os

## Set up the User Interface

In [ ]:
from IPython.display import display, HTML, clear_output, Markdown, FileLink
import textwrap, json
import ipywidgets as widgets
import re, time
import io
import PyPDF2
from PyPDF2 import PdfReader
from functools import partial
import os


In [ ]:

def print_wrapped(text):
    # Regular expression pattern to detect code blocks
    code_pattern = r'```(.+?)```'
    matches = list(re.finditer(code_pattern, text, re.DOTALL))
    if not matches:
        # If there are no code blocks, display the entire text as Markdown
        display(Markdown(text))
        return
    start = 0
    for match in matches:
        # Display the text before the code block as Markdown
        before_code = text[start:match.start()].strip()
        if before_code:
            display(Markdown(before_code))
        # Display the code block
        code = match.group(0).strip()  # Extract code block
        display(Markdown(code))  # Display code block
        start = match.end()
    # Display the text after the last code block as Markdown
    after_code = text[start:].strip()  # Text after the last code block
    if after_code:
        display(Markdown(after_code))

In [ ]:
DEFAULT_SYSTEM_PROMPT = f"""You are a helpful recipe-generating assistant. Based on the following given ingredients, you will generate a recipe. Make sure to follow the rules listed below: 1. Please don't give a very long recipe (more than 1000 words), make the description in 500-1000 words. 2. Warn the user if there is any common allergies ingredients in your recipe. 3. If you will need to use any ingredients outside of the ingredients that the user provided, Warn the user. 4. Provide other essential information about the recipe such as kitchen utensils, preparation steps. 5. Choose some common spice/sauce first, unless the user provided a very specific sauce want to use. 6. The default serving size is 2, unless the user specifies. 7. The default dish style is American/Italian cuisine, unless the user specifies. 8. The default type of dish is airfry/oven/stir-fry, unless the user specifies. 9. Use both text and some cute emoji if you can. """

SYSTEM_PROMPT = DEFAULT_SYSTEM_PROMPT


#initialize the dialog
dialog_history = [{"role": "system", "content": SYSTEM_PROMPT}]

button = widgets.Button(description="Send")

usertext = widgets.Textarea(layout=widgets.Layout(width='800px'))

output_log = widgets.Output()

#-------------------------------------------->
# Function to handle the subprocess output and update the dialog history
def generate_response(process, output_widget):
    while True:
        output = process.stdout.readline()
        #reinitialize assistant_response each time
        if process.poll() is not None and output == '':
            print("Subprocess has completed.")
            break 
        if output:
            if '[INST]' or '<>' in output :
                continue
            if '[/INST]' in output:
                inst_index = output.find('[/INST]')
                # Check if [/INST] is found in the text
                if inst_index != -1:
                    # Print everything after [/INST]
                    assistant_response = output[inst_index + len('[/INST]'):].strip()
            else:
                assistant_response = f"{output.strip()}"
            
            dialog_history.append({"role": "assistant", "content": assistant_response})
            
            if assistant_response:
                # Update the output widget
                with output_widget:
                    print_wrapped(f'{assistant_response}\n')
        else:
            break
    process.stdout.close()
#-------------------------------------------->  

#when the user start to use model
def on_button_clicked(b):
    user_input = usertext.value
    dialog_history.append({"role": "user", "content": user_input})
    usertext.value = ''

    # Change button description and color, and disable it
    button.description = 'Processing...'
    button.style.button_color = '#ff6e00'  # Use hex color codes for better color choices
    button.disabled = True  # Disable the button when processing

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

    prompt_template = f'''[INST] <<SYS>>
                        {SYSTEM_PROMPT}
                        <</SYS>>
                        {user_input} [/INST]'''
    
    # Start the subprocess and the threading to handle its output
    if (os.getcwd() != "/Users/astridz/Documents/AI_recipe/llama.cpp"):
        os.chdir('/Users/astridz/Documents/AI_recipe/llama.cpp')

    pure_name = "llama-2-7b-chat.Q4_K_M.gguf"
    args = ['./main', '-m', pure_name, '-c', '2048', '-ngl', '48', '-p', prompt_template]
    process = subprocess.Popen(args, stdout=subprocess.PIPE, text=True)
    # Start the thread that will handle the subprocess output
    output_thread = threading.Thread(target=generate_response, args=(process,output_log))
    output_thread.start()

    # Wait for the subprocess and thread to finish
    process.wait()
    output_thread.join()

    # Re-enable the button, reset description and color after processing
    button.description = 'Send'
    button.style.button_color = 'lightgray'
    button.disabled = False
    # os.getcwd() != "/Users/astridz/Documents/AI_recipe"


In [ ]:
button.on_click(on_button_clicked)

alert_out = widgets.Output()

clear_button = widgets.Button(description="Clear Chat")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

quit_button = widgets.Button(description="Force Quit")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

In [ ]:

def on_clear_button_clicked(b):
    # Clear the dialog history
    dialog_history.clear()
    # Add back the initial system prompt
    dialog_history.append({"role": "system", "content": SYSTEM_PROMPT})
    # Clear the output log
    with output_log:
        clear_output()
        
clear_button.on_click(on_clear_button_clicked)

from IPython.display import display, HTML
from ipywidgets import HBox, VBox

# Create the title with HTML
title = f"<h1 style='color: #ff6e00;'>Jupyter Recipe Llama 🦙 💻</h1> <p> Enter your ingredients! </p>"

# Assuming that output_log, alert_out, and text are other widgets or display elements...
first_row = HBox([button, clear_button, quit_button])  # Arrange these buttons horizontally

# Arrange the two rows of buttons and other display elements vertically
layout = VBox([output_log, alert_out, usertext, first_row])

display(HTML(title))  # Use HTML function to display the title
display(layout)
